In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from collections import Counter
import datetime
from scipy import stats
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Import Data

In [3]:
Feb7 = pd.read_csv('Feb_user_log_7.csv') 
Feb14 = pd.read_csv('Feb_user_log_14.csv') 
Feb30 = pd.read_csv('Feb_user_log.csv') 
Feb60 = pd.read_csv('Feb_user_log_60.csv') 
Feb7 = pd.read_csv('Feb_user_log_7.csv') 
Feb14 = pd.read_csv('Feb_user_log_14.csv') 
Feb30 = pd.read_csv('Feb_user_log.csv') 
Feb60 = pd.read_csv('Feb_user_log_60.csv') 

In [4]:
def memory_reduce_int(df):
    int_cols = list(df.select_dtypes(include=['int']).columns)
    for col in int_cols:
        if ((np.max(df[col]) <= 127)&(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767)&(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647)&(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
            
def memory_reduce_float(df):
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)
        
def memory_usage(df):
    memory = df.memory_usage(index=True).sum()
    print(memory/ 1024**2," MB")

def memory_reduce(df):
    memory_usage(df)
    memory_reduce_int(df)
    memory_reduce_float(df)
    memory_usage(df)

In [5]:
memory_reduce(Feb7)
memory_reduce(Feb14)
memory_reduce(Feb30)
memory_reduce(Feb60)
memory_reduce(Feb7)
memory_reduce(Feb14)
memory_reduce(Feb30)
memory_reduce(Feb60)

231.92990112304688  MB
115.96501159667969  MB
464.5681457519531  MB
232.2841339111328  MB
1040.8312683105469  MB
520.4156951904297  MB
2008.02197265625  MB
1004.0110473632812  MB
115.96501159667969  MB
115.96501159667969  MB
232.2841339111328  MB
232.2841339111328  MB
520.4156951904297  MB
520.4156951904297  MB
1004.0110473632812  MB
1004.0110473632812  MB


# Create features within 30days, 14 days and 14&30 interactions

In [6]:
Feb_30_group = Feb30.groupby(['msno'],as_index=False)
func = {'date':['count'], 
        'num_25':['sum'],'num_50':['sum'],
        'num_75':['sum'],'num_985':['sum'],
        'num_100':['sum'],'num_unq':['mean'],'total_secs':['sum','std']}
processed_Feb_30= Feb_30_group.agg(func)
processed_Feb_30.columns = processed_Feb_30.columns.get_level_values(0)
processed_Feb_30.columns.values[9] = 'secs_std' #30days std of listening seconds

processed_Feb_30['percentage_length'] = ((processed_Feb_30['num_25']*0.25+processed_Feb_30['num_50']*0.5
                                          +processed_Feb_30['num_75']*0.75+processed_Feb_30['num_985']*0.985
                                          +processed_Feb_30['num_100'])
                                         /(processed_Feb_30['num_25']+processed_Feb_30['num_50']
                                           +processed_Feb_30['num_75']+processed_Feb_30['num_985']
                                           +processed_Feb_30['num_100']))

processed_Feb_30['percent_whole_song'] = ((processed_Feb_30['num_985']+processed_Feb_30['num_100'])
                                          /(processed_Feb_30['num_25']+processed_Feb_30['num_50']
                                            +processed_Feb_30['num_75']+processed_Feb_30['num_985']
                                            +processed_Feb_30['num_100']))

processed_Feb_30['percent_unique'] = (processed_Feb_30['num_unq']
                                      /(processed_Feb_30['num_25']+processed_Feb_30['num_50']
                                        +processed_Feb_30['num_75']+processed_Feb_30['num_985']
                                        +processed_Feb_30['num_100']))
processed_Feb_30 = processed_Feb_30.rename(columns = {'date':'listening_days','num_unq':'unq_mean'})
processed_Feb_30 = processed_Feb_30.rename(columns=lambda s: s + '_30d')
processed_Feb_30.columns.values[0] = 'msno'

processed_Feb_30

,msno,listening_days_30d,num_25_30d,num_50_30d,num_75_30d,num_985_30d,num_100_30d,unq_mean_30d,total_secs_30d,secs_std_30d,percentage_length_30d,percent_whole_song_30d,percent_unique_30d
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,30,143,77,80,165,456.0,22.933333,169377.656250,3127.527832,0.817345,0.674267,0.024900
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,31,65,20,24,22,879.0,26.483871,224756.062500,7671.780273,0.935564,0.892079,0.026222
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,23,189,86,46,53,410.0,25.521739,131242.734375,5133.150879,0.748667,0.590561,0.032553
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,25,88,35,23,32,360.0,7.240000,108245.492188,3323.778809,0.833216,0.728625,0.013457
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,23,98,19,19,8,196.0,14.173913,56044.378906,3107.341553,0.741559,0.600000,0.041688
...,...,...,...,...,...,...,...,...,...,...,...,...,...
776169,zzx7wcys2Nha6yP2Skjs+R1/JNAfEuAWUaAp0qsV2Cg=,5,8,0,0,0,48.0,11.200000,11905.999023,1715.214600,0.892857,0.857143,0.200000
776170,zzxPS8+wCuE4HE85EDp4WBHgS5l548vfwXJbQq1HZ9I=,31,210,82,65,75,1374.0,50.709677,399372.625000,9125.776367,0.880468,0.802326,0.028078
776171,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,14,20,4,11,12,622.0,39.714286,166733.546875,8547.625000,0.970209,0.947683,0.059364
776172,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,16,105,21,17,22,153.0,15.625000,50209.339844,2579.602783,0.704937,0.550314,0.049135


In [7]:
Feb_14_group = Feb14.groupby(['msno'],as_index=False)
func = {'date':['count'], 
        'num_25':['sum'],'num_50':['sum'],
        'num_75':['sum'],'num_985':['sum'],
        'num_100':['sum'],'num_unq':['mean'],'total_secs':['sum','std']}
processed_Feb_14= Feb_14_group.agg(func)
processed_Feb_14.columns = processed_Feb_14.columns.get_level_values(0)
processed_Feb_14.columns.values[9] = 'secs_std'

processed_Feb_14['percentage_length'] = ((processed_Feb_14['num_25']*0.25+processed_Feb_14['num_50']*0.5
                                          +processed_Feb_14['num_75']*0.75+processed_Feb_14['num_985']*0.985
                                          +processed_Feb_14['num_100'])
                                         /(processed_Feb_14['num_25']+processed_Feb_14['num_50']
                                           +processed_Feb_14['num_75']+processed_Feb_14['num_985']
                                           +processed_Feb_14['num_100']))

processed_Feb_14['percent_whole_song'] = ((processed_Feb_14['num_985']+processed_Feb_14['num_100'])
                                          /(processed_Feb_14['num_25']+processed_Feb_14['num_50']
                                            +processed_Feb_14['num_75']+processed_Feb_14['num_985']
                                            +processed_Feb_14['num_100']))

processed_Feb_14['percent_unique'] = (processed_Feb_14['num_unq']
                                      /(processed_Feb_14['num_25']+processed_Feb_14['num_50']
                                        +processed_Feb_14['num_75']+processed_Feb_14['num_985']
                                        +processed_Feb_14['num_100']))
processed_Feb_14 = processed_Feb_14.rename(columns = {'date':'listening_days','num_unq':'unq_mean'})
processed_Feb_14 = processed_Feb_14.rename(columns=lambda s: s + '_14d')
processed_Feb_14.columns.values[0] = 'msno'
processed_Feb_14['secs_std_14d'].fillna(0, inplace=True)
processed_Feb_14

,msno,listening_days_14d,num_25_14d,num_50_14d,num_75_14d,num_985_14d,num_100_14d,unq_mean_14d,total_secs_14d,secs_std_14d,percentage_length_14d,percent_whole_song_14d,percent_unique_14d
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,13,64,35,32,69,217,22.307692,77573.375000,3114.801758,0.821259,0.685851,0.053496
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,14,12,8,7,7,513,34.142857,125873.882812,7374.789062,0.972843,0.950640,0.062418
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,9,84,37,19,14,128,25.444444,41406.074219,4157.230469,0.693404,0.503546,0.090229
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,11,61,30,19,18,153,10.000000,47477.179688,3259.900146,0.765943,0.608541,0.035587
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,9,41,13,10,4,124,20.333333,35314.125000,4632.727051,0.792656,0.666667,0.105903
...,...,...,...,...,...,...,...,...,...,...,...,...,...
733914,zzx7wcys2Nha6yP2Skjs+R1/JNAfEuAWUaAp0qsV2Cg=,3,2,0,0,0,32,11.333333,7703.554199,2224.673584,0.955882,0.941176,0.333333
733915,zzxPS8+wCuE4HE85EDp4WBHgS5l548vfwXJbQq1HZ9I=,14,66,34,18,26,684,53.571429,191729.062500,7999.591797,0.913780,0.857488,0.064700
733916,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,5,5,1,3,6,161,29.000000,44095.500000,9113.845703,0.971080,0.948864,0.164773
733917,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,8,34,10,3,9,84,13.875000,26198.376953,2776.315186,0.775821,0.664286,0.099107


In [9]:
Feb_1430 = processed_Feb_30.merge(processed_Feb_14,on='msno',how='left')

In [10]:
for colname in list(Feb_1430.columns):
    if Feb_1430[colname].isnull().values.any() == True:
        Feb_1430[colname].fillna(0, inplace=True)
    else: 
        pass

In [12]:
Feb_1430['1430_uniq_diff'] = Feb_1430['unq_mean_30d'] - Feb_1430['unq_mean_14d']
Feb_1430['1430_listen_diff'] = Feb_1430['listening_days_30d'] - Feb_1430['listening_days_14d']
Feb_1430['1430_perc_len_ratio'] = Feb_1430['percentage_length_14d']/Feb_1430['percentage_length_30d'] 

In [1]:
Feb_1430.columns

In [14]:
final1430 = Feb_1430.drop(columns=['num_25_30d', 'num_50_30d','num_75_30d','num_985_30d','num_100_30d','num_25_14d','num_50_14d','num_75_14d','num_985_14d','num_100_14d'])

# Create features in 60days and 30&60 interactions (recent & prior month)

In [15]:
Feb_60_group = Feb60.groupby(['msno'],as_index=False)
func = {'date':['count'], 
        'num_25':['sum'],'num_50':['sum'],
        'num_75':['sum'],'num_985':['sum'],
        'num_100':['sum'],'num_unq':['mean'],'total_secs':['sum','std']}
processed_Feb_60= Feb_60_group.agg(func)
processed_Feb_60.columns = processed_Feb_60.columns.get_level_values(0)
processed_Feb_60.columns.values[9] = 'secs_std' #30days std of listening seconds

processed_Feb_60['percentage_length'] = ((processed_Feb_60['num_25']*0.25+processed_Feb_60['num_50']*0.5
                                          +processed_Feb_60['num_75']*0.75+processed_Feb_60['num_985']*0.985
                                          +processed_Feb_60['num_100'])
                                         /(processed_Feb_60['num_25']+processed_Feb_60['num_50']
                                           +processed_Feb_60['num_75']+processed_Feb_60['num_985']
                                           +processed_Feb_60['num_100']))

processed_Feb_60['percent_whole_song'] = ((processed_Feb_60['num_985']+processed_Feb_60['num_100'])
                                          /(processed_Feb_60['num_25']+processed_Feb_60['num_50']
                                            +processed_Feb_60['num_75']+processed_Feb_60['num_985']
                                            +processed_Feb_60['num_100']))

processed_Feb_60['percent_unique'] = (processed_Feb_60['num_unq']
                                      /(processed_Feb_60['num_25']+processed_Feb_60['num_50']
                                        +processed_Feb_60['num_75']+processed_Feb_60['num_985']
                                        +processed_Feb_60['num_100']))
processed_Feb_60 = processed_Feb_60.rename(columns = {'date':'listening_days','num_unq':'unq_mean'})
processed_Feb_60 = processed_Feb_60.rename(columns=lambda s: s + '_60d')
processed_Feb_60.columns.values[0] = 'msno'

processed_Feb_60

,msno,listening_days_60d,num_25_60d,num_50_60d,num_75_60d,num_985_60d,num_100_60d,unq_mean_60d,total_secs_60d,secs_std_60d,percentage_length_60d,percent_whole_song_60d,percent_unique_60d
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,58,274,142,149,273,1048.0,25.500000,350597.187500,3482.105713,0.831471,0.700424,0.013521
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,60,142,53,50,142,1525.0,28.100000,411047.437500,6392.936035,0.922788,0.871862,0.014697
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,47,462,258,138,130,1014.0,34.106383,333475.625000,5016.520020,0.744281,0.571429,0.017036
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,52,159,54,40,51,826.0,9.153846,231112.062500,3740.079834,0.861049,0.776106,0.008101
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,47,152,38,31,26,388.0,12.744681,111301.929688,2456.543701,0.777732,0.651969,0.020070
...,...,...,...,...,...,...,...,...,...,...,...,...,...
798972,zzx7wcys2Nha6yP2Skjs+R1/JNAfEuAWUaAp0qsV2Cg=,10,10,0,0,0,94.0,9.900000,23329.802734,1722.773315,0.927885,0.903846,0.095192
798973,zzxPS8+wCuE4HE85EDp4WBHgS5l548vfwXJbQq1HZ9I=,60,488,213,131,159,2579.0,51.066667,754627.812500,8831.558594,0.857805,0.766947,0.014304
798974,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,24,27,7,18,15,770.0,29.375000,208091.687500,7834.630859,0.965980,0.937873,0.035096
798975,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,19,112,21,18,24,161.0,14.000000,52916.968750,2530.030518,0.704286,0.550595,0.041667


In [16]:
final_60 = processed_Feb_60.iloc[: , [0,1,7,8,9,10,11,12]].copy() 

In [17]:
Feb60=Feb60[Feb60['msno'].isin(Feb30['msno'])]

In [18]:
Feb3060=Feb60[~Feb60.date.isin(Feb30.date)]

In [19]:
def process(data):
    data_group=data.groupby(['msno'],as_index=False)
    func = {'date':['count'], 
        'num_25':['sum'],'num_50':['sum'],
        'num_75':['sum'],'num_985':['sum'],
        'num_100':['sum'],'num_unq':['sum'],'total_secs':['sum']}
    processed_data=data_group.agg(func)
    processed_data.columns=processed_data.columns.get_level_values(0)
    
    processed_data['percentage_length'] = ((processed_data['num_25']*0.25+processed_data['num_50']*0.5
                                          +processed_data['num_75']*0.75+processed_data['num_985']*0.985
                                          +processed_data['num_100'])
                                         /(processed_data['num_25']+processed_data['num_50']
                                           +processed_data['num_75']+processed_data['num_985']
                                           +processed_data['num_100']))

    processed_data['percent_whole_song'] = ((processed_data['num_985']+processed_data['num_100'])
                                          /(processed_data['num_25']+processed_data['num_50']
                                            +processed_data['num_75']+processed_data['num_985']
                                            +processed_data['num_100']))

    processed_data['percent_unique'] = (processed_data['num_unq']
                                      /(processed_data['num_25']+processed_data['num_50']
                                        +processed_data['num_75']+processed_data['num_985']
                                        +processed_data['num_100']))
    
    processed_data['percent_25'] = (processed_data['num_25']
                                      /(processed_data['num_25']+processed_data['num_50']
                                        +processed_data['num_75']+processed_data['num_985']
                                        +processed_data['num_100']))
    
    
    processed_data=processed_data.rename(columns={'date':'listening_days','num_unq':'unq_mean'})
    return processed_data


In [20]:
prior_Feb3060=process(Feb3060)
prior_Feb_30=process(Feb30)

In [22]:
prior_Feb_30['quick_churn']=~prior_Feb_30.msno.isin(prior_Feb3060.msno)*1
prior_Feb_30=prior_Feb_30.add_prefix('30d_')
prior_Feb_30=prior_Feb_30.rename(columns={'30d_msno':'msno'})
prior_Feb_30=prior_Feb_30.rename(columns={'30d_quick_churn':'quick_churn'})

In [23]:
prior_Feb3060=prior_Feb3060.add_prefix('3060_')
prior_Feb3060=prior_Feb3060.rename(columns={'3060_msno':'msno'})

In [24]:
prior_Feb_30=prior_Feb_30.merge(prior_Feb3060,on='msno',how='left')#.fillna(0)

In [25]:
# for colname in list(prior_Feb_30.columns):
#     if prior_Feb_30[colname].isnull().values.any() == True:
#         prior_Feb_30[colname].fillna((prior_Feb_30[colname].mean()), inplace=True)
#     else: 
#         pass

In [26]:
prior_Feb_30['3060_listening_days'].fillna(0, inplace=True)

In [27]:
prior_Feb_30['3060_listen_days_diff']=prior_Feb_30['30d_listening_days']- prior_Feb_30['3060_listening_days']
prior_Feb_30['3060_total_seconds_ratio']=prior_Feb_30['30d_total_secs'].div(prior_Feb_30['3060_total_secs'], level=1)
prior_Feb_30['3060_total_seconds_ratio'].fillna((prior_Feb_30['3060_total_seconds_ratio'].mean()), inplace=True)
prior_Feb_30['3060_perc_len_ratio']=prior_Feb_30['30d_percentage_length'].div(prior_Feb_30['3060_percentage_length'], level=1)
prior_Feb_30['3060_perc_len_ratio'].fillna((prior_Feb_30['3060_perc_len_ratio'].mean()), inplace=True)
prior_Feb_30['3060_perc_100_ratio']=prior_Feb_30['30d_percent_whole_song'].div(prior_Feb_30['3060_percent_whole_song'], level=1)
prior_Feb_30['3060_perc_100_ratio'] = prior_Feb_30['3060_perc_100_ratio'].replace(np.inf,np.nan)
prior_Feb_30['3060_perc_100_ratio'].fillna((prior_Feb_30['3060_perc_100_ratio'].mean()), inplace=True)
prior_Feb_30['3060_uniq_diff']=prior_Feb_30['30d_unq_mean']- prior_Feb_30['3060_unq_mean']
prior_Feb_30['3060_uniq_diff'].fillna((prior_Feb_30['3060_uniq_diff'].mean()), inplace=True)

In [33]:
final_3060 = prior_Feb_30.iloc[: , [0,12,13,26,27,28,29,30]].copy() 

# Adding features from 7days dataset

In [34]:
Feb_7_group = Feb7.groupby(['msno'],as_index=False)
func = {'date':['count'], 
        'num_25':['sum'],'num_50':['sum'],
        'num_75':['sum'],'num_985':['sum'],
        'num_100':['sum'],'num_unq':['mean'],'total_secs':['sum','std']}
processed_Feb_7= Feb_7_group.agg(func)
processed_Feb_7.columns = processed_Feb_7.columns.get_level_values(0)
processed_Feb_7.columns.values[9] = 'secs_std' #7days std of listening seconds

processed_Feb_7['percentage_length'] = ((processed_Feb_7['num_25']*0.25+processed_Feb_7['num_50']*0.5
                                          +processed_Feb_7['num_75']*0.75+processed_Feb_7['num_985']*0.985
                                          +processed_Feb_7['num_100'])
                                         /(processed_Feb_7['num_25']+processed_Feb_7['num_50']
                                           +processed_Feb_7['num_75']+processed_Feb_7['num_985']
                                           +processed_Feb_7['num_100']))

processed_Feb_7['percent_whole_song'] = ((processed_Feb_7['num_985']+processed_Feb_7['num_100'])
                                          /(processed_Feb_7['num_25']+processed_Feb_7['num_50']
                                            +processed_Feb_7['num_75']+processed_Feb_7['num_985']
                                            +processed_Feb_7['num_100']))

processed_Feb_7['percent_unique'] = (processed_Feb_7['num_unq']
                                      /(processed_Feb_7['num_25']+processed_Feb_7['num_50']
                                        +processed_Feb_7['num_75']+processed_Feb_7['num_985']
                                        +processed_Feb_7['num_100']))
processed_Feb_7 = processed_Feb_7.rename(columns = {'date':'listening_days','num_unq':'unq_mean'})
processed_Feb_7 = processed_Feb_7.rename(columns=lambda s: s + '_7d')
processed_Feb_7.columns.values[0] = 'msno'

processed_Feb_7

,msno,listening_days_7d,num_25_7d,num_50_7d,num_75_7d,num_985_7d,num_100_7d,unq_mean_7d,total_secs_7d,secs_std_7d,percentage_length_7d,percent_whole_song_7d,percent_unique_7d
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,7,40,22,18,50,133,25.285714,48807.703125,3094.755371,0.824144,0.695817,0.096143
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,7,7,4,4,4,380,52.428571,92843.007812,7930.931641,0.979173,0.962406,0.131400
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,5,48,14,14,11,49,21.400000,17725.072266,3435.660889,0.656875,0.441176,0.157353
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,7,39,16,10,8,77,9.428571,22343.583984,2003.293091,0.734200,0.566667,0.062857
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,5,18,7,6,3,92,23.600000,25711.365234,5875.253906,0.852817,0.753968,0.187302
...,...,...,...,...,...,...,...,...,...,...,...,...,...
682653,zzx4hKiyR9XFEGAr7SAjcCPbKJCZ+IqegWL7dPjPwZk=,1,1,0,0,1,22,24.000000,5490.765137,NaN,0.968125,0.958333,1.000000
682654,zzx7wcys2Nha6yP2Skjs+R1/JNAfEuAWUaAp0qsV2Cg=,2,2,0,0,0,18,10.000000,4048.798096,2850.666992,0.925000,0.900000,0.500000
682655,zzxPS8+wCuE4HE85EDp4WBHgS5l548vfwXJbQq1HZ9I=,7,40,24,11,15,404,63.000000,113549.898438,7147.636719,0.908957,0.848178,0.127530
682656,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,3,3,1,3,5,145,42.000000,39480.601562,9682.500977,0.977229,0.955414,0.267516


In [35]:
final7 = processed_Feb_7.iloc[: , [0,1,7,8,9,10,11,12]].copy() 

# Combine and output data

In [36]:
final1 = final1430.merge(final_3060,on='msno',how='left')#.fillna(0)
final2 = final1.merge(final_60,on='msno',how='left')
final_days = final2.merge(final7,on='msno',how='left').fillna(0)

In [240]:
#final_days.to_csv('final_days')

In [37]:
Feb_total = pd.read_csv('Feb_total.csv') 

In [38]:
Final_log_Feb = final_days.merge(Feb_total,on='msno',how='left')

In [39]:
Final_log_Feb

,msno,listening_days_30d,unq_mean_30d,total_secs_30d,secs_std_30d,percentage_length_30d,percent_whole_song_30d,percent_unique_30d,listening_days_14d,unq_mean_14d,...,num_100_sum,num_unq_sum,total_secs_sum,membership_expire_date,last_day,percentage_length,percent_whole_song,percent_lower25_song,percent_unique,std_sec
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,30,22.933333,169377.656250,3127.527832,0.817345,0.674267,0.024900,13.0,22.307692,...,1980,2453,6.409221e+05,2017-02-15,1,0.829631,0.700862,0.149425,0.704885,1.370140e+07
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,31,26.483871,224756.062500,7671.780273,0.935564,0.892079,0.026222,14.0,34.142857,...,15380,15080,4.012320e+06,2017-02-19,1,0.923625,0.874732,0.075897,0.828162,4.545468e+07
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,23,25.521739,131242.734375,5133.150879,0.748667,0.590561,0.032553,9.0,25.444444,...,23749,24559,6.662899e+06,2017-02-26,1,0.817318,0.709049,0.187717,0.692525,1.144109e+08
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,25,7.240000,108245.492188,3323.778809,0.833216,0.728625,0.013457,11.0,10.000000,...,4987,3780,1.454415e+06,2017-02-15,1,0.821992,0.714999,0.183346,0.496454,3.107733e+07
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,23,14.173913,56044.378906,3107.341553,0.741559,0.600000,0.041688,9.0,20.333333,...,4780,6599,1.392712e+06,2017-02-23,1,0.837467,0.731381,0.151485,0.929044,1.683024e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776169,zzx7wcys2Nha6yP2Skjs+R1/JNAfEuAWUaAp0qsV2Cg=,5,11.200000,11905.999023,1715.214600,0.892857,0.857143,0.200000,3.0,11.333333,...,2367,2310,6.353585e+05,2017-02-09,3,0.883463,0.815922,0.123950,0.775949,5.149372e+06
776170,zzxPS8+wCuE4HE85EDp4WBHgS5l548vfwXJbQq1HZ9I=,31,50.709677,399372.625000,9125.776367,0.880468,0.802326,0.028078,14.0,53.571429,...,63057,57588,1.594633e+07,2017-02-10,1,0.961208,0.936733,0.037421,0.844758,5.158133e+08
776171,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,14,39.714286,166733.546875,8547.625000,0.970209,0.947683,0.059364,5.0,29.000000,...,30470,25448,8.102517e+06,2017-02-26,1,0.939030,0.903787,0.065995,0.740823,1.768040e+08
776172,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,16,15.625000,50209.339844,2579.602783,0.704937,0.550314,0.049135,8.0,13.875000,...,2332,2687,6.865855e+05,2017-02-24,2,0.812487,0.705680,0.197648,0.770797,1.052606e+07


In [40]:
Final_log_Feb.to_csv('Final_log_Feb.csv')